## Motivation 

"Sunshine, temperature, and rain have a significant impact on dailysales, particularly in the summer, on weekends, and on days with extreme weather." 

"Using weather forecasts, we have sig-nificantly improved sales forecast accuracy. We find that including weather data in the sales forecast model can lead tofewer sales forecast errors, reducing them by, on average, 8.6% to 12.2% and up to 50.6% on summer weekends"


**Goal**

1. Get Temperature Data 
2. Get Caily sunshine cat data (sunshine, cloudy, raining)
    - raining includes fog, snow and heavy 


In [15]:
# Libraries

# Basics
import pandas as pd 
import numpy as np 
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

# Wetterdiesnt, API for deutscher wetterdienst (dwd)
from wetterdienst.provider.dwd.observation import DwdObservationRequest, \
    DwdObservationPeriod, DwdObservationResolution, DwdObservationParameter, DwdObservationDataset


## Wetterdienst

#### Get all Parameters

In [ ]:
# all
print("All available parameters")
print(
    DwdObservationRequest.discover()
)
 selection
 print("Selection of daily data")
print(
    DwdObservationRequest.discover(
        filter_=DwdObservationResolution.DAILY
    )
)



List of variables I want:

sunshine_duration (in min)
TEMPERATURE_AIR_MEAN_200
temperature_air_min_200
temperature_air_max_200



Select weather station per Bundesland:

### Weather per state code

In [ ]:
# Function that extracts average temperature, average percipitation and sunshine duration from dwd api 

def get_weather_data(start_date, end_date, station_id):
    
    request = DwdObservationRequest(
    parameter=[
         DwdObservationParameter.DAILY.TEMPERATURE_AIR_MEAN_200,
        DwdObservationParameter.DAILY.PRECIPITATION_HEIGHT,
        DwdObservationParameter.DAILY["SUNSHINE_DURATION"]
    ],
    resolution=DwdObservationResolution.DAILY,
    start_date=start_date,  # if not given timezone defaulted to UTC
    end_date=end_date
    #period=DwdObservationPeriod.HISTORICAL
    ).filter_by_station_id(station_id=station_id)
    
    station_data = request.values.all().df
    
    return station_data

In [ ]:
# Create 16 datasets per state in germany. name datasets according to state name 

# Data Manually colelcted 
cities = ['Hamburg-Fuhlsbüttel', 'Schleswig', 'Bremen', 'Berlin-Dahlem (FU)', 'Potsdam', 'Schwerin', 'Hannover', 
          'Dresden-Klotzsche', 'Magdeburg', 'Erfurt-Weimar', 'Düsseldorf', 'Stuttgart-Echterdingen', 'Frankfurt/Main', 
          'Saarbrücken-Ensheim', 'Trier-Petrisberg', 'München-Stadt']

station_id = [1975, 4466, 691, 403, 3987, 4625, 2014, 1048, 3126, 1270, 1078, 4931, 1420, 4336, 5100, 3379]

# List all available stations
request = DwdObservationRequest(
    parameter=DwdObservationDataset.PRECIPITATION_MORE,
    resolution=DwdObservationResolution.DAILY,
    period=DwdObservationPeriod.HISTORICAL
)

# Save df 
stations = request.all().df

# Select stations based on station names that are saved in "cities"
selected_stations = stations[stations.name.isin(cities)]

# Change names of station names to avoid errors
selected_stations['state'] = selected_stations['state'].str.replace('-','_')
selected_stations['state'] = selected_stations['state'].str.replace('ü','ue')
selected_stations['state'] = selected_stations['state'].str.replace('/','_')

selected_stations.reset_index(drop=True)
    

# Create Data Frames 

# The code selects the statename from the data frame selected stations and assigns the corresponding dataframe based on station_id 
for i in range(16):
    name = selected_stations["state"].tolist()[i]
    exec(f"{name} = get_weather_data('2015-12-30', '2022-12-31', ([int(x) for x in selected_stations['station_id'].tolist()][i], ))")

#### Create Features 

1. 3 columns for each parameter (temperature_air_mean_200, sunshine_duration, percipitation (rain))
2. Temperature: scale 1-10 for each month average (look at paper)
3. Sunshine -> convert into hours /3600
4. Percipitation (look into paper, 0 for no rain -> 1-10 percentiles)

In [ ]:
# Create 3 columns 

def pivot_weather(df):
    df_pivot = df.pivot_table(index=["date","state","station_id"], columns= ["parameter"], values="value").reset_index()
    df_pivot["station_id"] = df["station_id"]
    #df_pivot["state"] = df["state"]
    return df_pivot 

# test
#Brandenburg["state"] = "Brandenburg"
#Brandenburg_p = pivot_weather(Brandenburg)
#Brandenburg_p

In [ ]:
# Sunshine 

# Change class borders to allow 10 classes
def jitter(a_series, noise_reduction=1000000):
    return (np.random.random(len(a_series))*a_series.std()/noise_reduction)-(a_series.std()/(2*noise_reduction))

def sunshine_hours(df):
    df["sunshine_duration_h"] = df["sunshine_duration"]/3600
    
    df["suns_classes"] = df.groupby(df["date"].dt.month)["sunshine_duration_h"].transform(
    lambda x: pd.qcut(x + jitter(x), 10, labels=range(0,10)))
    
    df.sunshine_duration.drop
    
    return df

# test
#sunshine_hours(Brandenburg_p)

To make the different months more comparable, we compute relative weather classes, which we refer to as scores. More specifically, we build monthly deciles using data from 1997 to 2012. The weather score expresses the relative classification of a weather parameter on 1 day compared with all of the days in that month across the 15-year estimation sample.

In [ ]:
# Temperature

def temp_classes(df):
    df["temp_classes"] = df.groupby(df["date"].dt.month)["temperature_air_mean_200"].transform(
                     lambda x: pd.qcut(x, 10, labels=range(0,10)))
    return df
    
# test
#temp_classes(Brandenburg_p).head(5)


In [ ]:
# Rain 

# introduce a minimal amount of noise, which will artificially create unique bin edges. (source: stackoverflow)
def jitter(a_series, noise_reduction=1000000):
    return (np.random.random(len(a_series))*a_series.std()/noise_reduction)-(a_series.std()/(2*noise_reduction))


def rain_classes(df):
    
    # Zero Values are NaN now, so qcut does ignore these values (more than 50% of values are 0)
    df["precipitation_height"].replace(to_replace=0, value=np.nan, inplace=True)
    
    # Create monthly percentiles bases on remaining values 
    df["rain_classes"] = df.groupby(df["date"].dt.month)["precipitation_height"].transform(
    lambda x: pd.qcut(x + jitter(x), 10, labels=range(0,10)))
    
    # Fill Na Values with zeros 
    df['rain_classes'] = df['rain_classes'].fillna(0)
    df["precipitation_height"] =  df["precipitation_height"].fillna(0)
    
    return df

# test
#rain_classes(Brandenburg_p).head(5)

In [ ]:
# Summarise 
def all_weather_features(df):
    
    # Create 3 columns 
    df_p = pivot_weather(df)
    
    # fillna with mean values 
    columns_with_na = df_p.columns[df_p.isna().any()].tolist()
    df_p[columns_with_na] = df_p[columns_with_na].apply(lambda x: x.fillna(x.mean()),axis=0)
    
    # Sunshine
    df_p = sunshine_hours(df_p)
    # Temperature
    df_p = temp_classes(df_p)
    # Rain
    df_p = rain_classes(df_p)
    
    return df_p

#test
#Brandenburg_p = all_weather_features(Brandenburg)
#Brandenburg

In [ ]:
# Get everything together

# Names of all dfs
dfs = [Brandenburg, Berlin, Bremen, Sachsen, Nordrhein_Westfalen, Thueringen, Hessen, Hamburg, Niedersachsen, Sachsen_Anhalt, Bayern, Saarland, Schleswig_Holstein, Mecklenburg_Vorpommern, Baden_Wuerttemberg, Rheinland_Pfalz]

# Empty Dictionary
df_dict = {}

#
for df, u in zip(dfs, selected_stations["state"].tolist()):
    # Extract the name of the data frame
    name = u
    # Add the data frame to the dictionary with the name as the key
    df_dict[name] = df
    
#df_dict

for key, df in df_dict.items():
    df_dict[key] = df.assign(state=key)

#Finally, to concatenate all of the data frames together, I use concat function and pass in the list of data frames as the argument:
df_concat = pd.concat([df for df in df_dict.values()]).reset_index()

state_df = all_weather_features(df_concat)

# Make station_id, suns_classes, temp_classes and rain_classes an integer
state_df[['station_id', 
    'suns_classes', 
    'temp_classes', 
    'rain_classes']] = state_df[['station_id', 
                           'suns_classes', 
                           'temp_classes', 
                           'rain_classes']].astype(int)

In [ ]:
# Create summary for Germany 

# Groupby mean and date 
germany_df = state_df.groupby("date").mean()
germany_df["state"] = "Deutschland"
germany_df["station_id"] = 99999

# Create weather classes with same method as before

# Sunshine
germany_df = sunshine_hours(germany_df.reset_index())
# Temperature
germany_df = temp_classes(germany_df)
# Rain
germany_df = rain_classes(germany_df)

germany_df[['station_id', 
    'suns_classes', 
    'temp_classes', 
    'rain_classes']] = germany_df[['station_id', 
                           'suns_classes', 
                           'temp_classes', 
                           'rain_classes']].astype(int)

In [ ]:
final_df = state_df.merge(germany_df, how="outer")

In [ ]:
# Save df as csv

final_df.set_index("date").to_csv("../data/intermediate/weather.csv")

# test
#pd.read_csv("../data/intermediate/weather.csv").info()